
# Proyecto: Aumento de Imágenes y CNN con **Sign Language MNIST** (CSV)

**Objetivo**  
- Cargar el dataset _Sign Language MNIST_ desde CSV.  
- Aplicar **≥ 6 transformaciones** (rotación, traslación, escalado, inversión, ruido, recorte aleatorio…).  
- Implementar una **CNN en PyTorch** (≥ 2 capas `Conv2d` y capas de `Pooling`) explorando **kernel size, stride, padding y pooling**.  
- Entrenar durante **≥ 5 épocas** con **Adam**, mostrando **pérdida y precisión** en **train** y **valid** tras cada época.  
- Desarrollar todo en este **Notebook** para subirlo a GitHub.



## Requisitos previos

1. Descarga desde Kaggle el dataset **Sign Language MNIST** en CSV (p. ej. `sign_mnist_train.csv` y `sign_mnist_test.csv`).  
2. Sube el/los CSV(s) a la carpeta del entorno donde ejecutes este Notebook (Colab / local).  
3. Si estás en Windows y `num_workers=2` da problemas, pon `num_workers=0`.


link para descargar dataset -> https://www.kaggle.com/datasets/datamunge/sign-language-mnist

In [1]:

# Imports principales
import os
import math
import time
import random
import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, random_split

import torchvision
from torchvision import transforms

import matplotlib.pyplot as plt

# Config reproducibilidad
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
DEVICE


device(type='cpu')


## Parte 1 · Carga desde CSV (imágenes 28×28 en escala de grises)

Cada fila del CSV contiene una imagen 28×28 (784 columnas) + una columna `label` con la clase (A–Y excepto J y Z en `train`).  
Transformaremos los píxeles a `float32` y normalizaremos a \[0, 1].


In [2]:

# Ruta a los CSV (ajusta si hace falta)
TRAIN_CSV = 'dataset/sign_mnist_train.csv'  # cambiar si tu archivo tiene otro nombre
TEST_CSV  = 'datset/sign_mnist_test.csv'   # opcional para evaluación extra

assert os.path.exists(TRAIN_CSV), f"No se encontró {TRAIN_CSV}. Sube el CSV a esta carpeta."
print("CSV de entrenamiento encontrado.")

df_train = pd.read_csv(TRAIN_CSV)
print(df_train.head())

labels = df_train['label'].values.astype(np.int64)
X = df_train.drop(columns='label').to_numpy().astype(np.float32)
# Reescalar a [0,1]
X = X / 255.0
# Reformar a (N, 1, 28, 28)
X = X.reshape(-1, 1, 28, 28)
X.shape, labels.shape


CSV de entrenamiento encontrado.
   label  pixel1  pixel2  pixel3  pixel4  pixel5  pixel6  pixel7  pixel8  \
0      3     107     118     127     134     139     143     146     150   
1      6     155     157     156     156     156     157     156     158   
2      2     187     188     188     187     187     186     187     188   
3      2     211     211     212     212     211     210     211     210   
4     13     164     167     170     172     176     179     180     184   

   pixel9  ...  pixel775  pixel776  pixel777  pixel778  pixel779  pixel780  \
0     153  ...       207       207       207       207       206       206   
1     158  ...        69       149       128        87        94       163   
2     187  ...       202       201       200       199       198       199   
3     210  ...       235       234       233       231       230       226   
4     185  ...        92       105       105       108       133       163   

   pixel781  pixel782  pixel783  pixel784

((27455, 1, 28, 28), (27455,))


## Selección de una imagen y **≥ 6 transformaciones**

Aplicamos transformaciones de `torchvision.transforms` y una transformación personalizada de **ruido gaussiano**.  
Mostramos la imagen original junto a 6 versiones transformadas.


In [3]:

# Selecciona una imagen (por ejemplo la primera)
idx = 0
img0 = X[idx]  # (1, 28, 28) en [0,1]
label0 = labels[idx]
print("Etiqueta de la imagen seleccionada:", label0)

# Definimos 6+ transformaciones (sobre PIL o Tensor). Usaremos PIL para compatibilidad amplia:
to_pil = transforms.ToPILImage()
to_tensor = transforms.ToTensor()

class AddGaussianNoise(object):
    def __init__(self, mean=0.0, std=0.1):
        self.mean = mean
        self.std = std
    def __call__(self, tensor):
        noise = torch.randn_like(tensor) * self.std + self.mean
        out = tensor + noise
        return torch.clamp(out, 0.0, 1.0)

# Conjunto de transformaciones sugeridas
tfs = [
    transforms.RandomRotation(degrees=25),
    transforms.RandomAffine(degrees=0, translate=(0.2, 0.2)),          # traslación
    transforms.RandomAffine(degrees=0, scale=(0.7, 1.3)),              # escalado
    transforms.RandomHorizontalFlip(p=1.0),                             # espejo horizontal
    transforms.RandomInvert(p=1.0),                                     # inversión de color
    transforms.RandomCrop(size=24, padding=2),                          # recorte aleatorio
]

# Añadimos ruido gaussiano como transform tensor-based
noise_tf = transforms.Compose([to_tensor, AddGaussianNoise(0.0, 0.2)])
# Para el resto (PIL-based): ToPIL -> tf -> ToTensor
def apply_pil_tf(img_tensor, pil_tf):
    pil = to_pil(img_tensor)      # (1,28,28) -> PIL (L)
    out = pil_tf(pil)             # PIL -> PIL
    return to_tensor(out)         # PIL -> (1,H,W) [0,1]

# Generar versiones transformadas
transformed = []
# 5 PIL-based
for tf in tfs[:5]:
    transformed.append(apply_pil_tf(img0, tf))
# RandomCrop también es PIL-based
transformed.append(apply_pil_tf(img0, tfs[5]))
# Ruido (tensor-based)
transformed.append(noise_tf(img0))

len(transformed)


Etiqueta de la imagen seleccionada: 3


ValueError: pic should not have > 4 channels. Got 28 channels.

In [ ]:

# Visualización: original + 6 transformaciones (7 imágenes)
fig, axes = plt.subplots(1, 7, figsize=(18, 3))
axes = axes.flatten()

axes[0].imshow(img0.squeeze(), cmap='gray')
axes[0].set_title("Original")
axes[0].axis('off')

titles = ["Rotación", "Traslación", "Escalado", "Flip H", "Invertir", "Crop", "Ruido"]
for i, (ax, timg, title) in enumerate(zip(axes[1:], transformed, titles)):
    ax.imshow(timg.squeeze(), cmap='gray')
    ax.set_title(title)
    ax.axis('off')

plt.tight_layout()
plt.show()



## Parte 2 · Dataset, Split y DataLoader

- Convertimos `X, y` a tensores.  
- Split **80/20** en _train/valid_.  
- `DataLoader` con `batch_size=32` y `num_workers=2` (ajusta a `0` si Windows da problemas).


In [ ]:

# Tensores
X_tensor = torch.from_numpy(X)            # (N,1,28,28), float32
y_tensor = torch.from_numpy(labels)       # (N,), int64

class SLMDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    def __len__(self):
        return self.X.shape[0]
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

full_ds = SLMDataset(X_tensor, y_tensor)

# Split 80/20
N = len(full_ds)
n_train = int(0.8 * N)
n_val = N - n_train
train_ds, val_ds = random_split(full_ds, [n_train, n_val], generator=torch.Generator().manual_seed(SEED))

BATCH_SIZE = 32
NUM_WORKERS = 2  # si en Windows falla, usa 0

train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,  num_workers=NUM_WORKERS, pin_memory=True)
val_loader   = DataLoader(val_ds,   batch_size=BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS, pin_memory=True)

len(train_ds), len(val_ds)



## Exploración de tamaños de salida en **Conv2d** y **Pooling**

Función auxiliar para calcular tamaños de salida al variar **kernel_size**, **stride**, **padding** y **pooling**.  
Fórmula para una dimensión:  
\[ \text{out} = \left\lfloor \frac{\text{in} + 2\,\text{padding} - \text{kernel}}{\text{stride}} \right\rfloor + 1 \]


In [ ]:

def conv_out_size(in_size, kernel, stride=1, padding=0):
    return (in_size + 2*padding - kernel)//stride + 1

def pool_out_size(in_size, kernel, stride=None, padding=0):
    if stride is None:
        stride = kernel
    return (in_size + 2*padding - kernel)//stride + 1

# Ejemplos de combinaciones
input_h = input_w = 28
for k in [3,5]:
    for p in [0,1]:
        for s in [1,2]:
            out_h = conv_out_size(input_h, k, s, p)
            out_w = conv_out_size(input_w, k, s, p)
            print(f"Conv: kernel={k}, padding={p}, stride={s} -> out=({out_h}x{out_w})")



## Implementación de la CNN (parametrizable)

- **2+ capas convolucionales** con opciones de `kernel_size`, `stride`, `padding`.  
- **Pooling** configurable.  
- Capa fully-connected para clasificación (25 clases en `train`: letras A–Y excepto J y Z).


In [ ]:

NUM_CLASSES = len(np.unique(labels))
print("NUM_CLASSES:", NUM_CLASSES)

class SimpleCNN(nn.Module):
    def __init__(self, 
                 c1_out=32, c2_out=64, 
                 k1=3, s1=1, p1=1,
                 k2=3, s2=1, p2=1,
                 pool_kernel=2, pool_stride=2,
                 use_pool=True):
        super().__init__()
        self.use_pool = use_pool

        self.conv1 = nn.Conv2d(1, c1_out, kernel_size=k1, stride=s1, padding=p1)
        self.conv2 = nn.Conv2d(c1_out, c2_out, kernel_size=k2, stride=s2, padding=p2)
        self.pool  = nn.MaxPool2d(kernel_size=pool_kernel, stride=pool_stride)

        # Calcular tamaño tras conv/pool para definir la FC
        h = w = 28
        h = conv_out_size(h, k1, s1, p1)
        w = conv_out_size(w, k1, s1, p1)
        if use_pool:
            h = pool_out_size(h, pool_kernel, pool_stride)
            w = pool_out_size(w, pool_kernel, pool_stride)

        h = conv_out_size(h, k2, s2, p2)
        w = conv_out_size(w, k2, s2, p2)
        if use_pool:
            h = pool_out_size(h, pool_kernel, pool_stride)
            w = pool_out_size(w, pool_kernel, pool_stride)

        feat_dim = c2_out * h * w
        self.head = nn.Linear(feat_dim, NUM_CLASSES)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        if self.use_pool:
            x = self.pool(x)
        x = F.relu(self.conv2(x))
        if self.use_pool:
            x = self.pool(x)
        x = torch.flatten(x, 1)
        x = self.head(x)
        return x

# Ejemplo de instanciación (padding=1, stride=1, kernel=3; pooling 2x2)
model = SimpleCNN(k1=3, s1=1, p1=1, k2=3, s2=1, p2=1, pool_kernel=2, pool_stride=2, use_pool=True).to(DEVICE)
sum(p.numel() for p in model.parameters()), model



## Entrenamiento (≥ 5 épocas) y evaluación (pérdida y precisión)

- Optimizador **Adam**.  
- Reporte de **loss** y **accuracy** en **train** y **valid** por época.  
- Ajusta `EPOCHS`, `lr` y la configuración de la CNN para experimentar.


In [ ]:

def accuracy_from_logits(logits, y_true):
    preds = logits.argmax(dim=1)
    return (preds == y_true).float().mean().item()

def train_one_epoch(model, loader, optimizer, loss_fn, device):
    model.train()
    running_loss, running_acc, n = 0.0, 0.0, 0
    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)

        optimizer.zero_grad()
        logits = model(xb)
        loss = loss_fn(logits, yb)
        loss.backward()
        optimizer.step()

        bsz = yb.size(0)
        running_loss += loss.item() * bsz
        running_acc  += accuracy_from_logits(logits, yb) * bsz
        n += bsz
    return running_loss / n, running_acc / n

@torch.no_grad()
def evaluate(model, loader, loss_fn, device):
    model.eval()
    running_loss, running_acc, n = 0.0, 0.0, 0
    for xb, yb in loader:
        xb = xb.to(device, non_blocking=True)
        yb = yb.to(device, non_blocking=True)
        logits = model(xb)
        loss = loss_fn(logits, yb)
        bsz = yb.size(0)
        running_loss += loss.item() * bsz
        running_acc  += accuracy_from_logits(logits, yb) * bsz
        n += bsz
    return running_loss / n, running_acc / n

EPOCHS = 5
lr = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

history = {"train_loss": [], "train_acc": [], "val_loss": [], "val_acc": []}

start = time.time()
for epoch in range(1, EPOCHS+1):
    tr_loss, tr_acc = train_one_epoch(model, train_loader, optimizer, loss_fn, DEVICE)
    va_loss, va_acc = evaluate(model, val_loader, loss_fn, DEVICE)

    history["train_loss"].append(tr_loss)
    history["train_acc"].append(tr_acc)
    history["val_loss"].append(va_loss)
    history["val_acc"].append(va_acc)

    print(f"Epoch {epoch:02d} | "
          f"Train Loss: {tr_loss:.4f}  Acc: {tr_acc:.4f} | "
          f"Val Loss: {va_loss:.4f}  Acc: {va_acc:.4f}")
elapsed = time.time() - start
print(f"Tiempo total de entrenamiento: {elapsed/60:.2f} min")


In [ ]:

# Curvas de pérdida y precisión
epochs = range(1, EPOCHS+1)

plt.figure(figsize=(6,4))
plt.plot(epochs, history["train_loss"], label="Train Loss")
plt.plot(epochs, history["val_loss"],   label="Val Loss")
plt.xlabel("Época")
plt.ylabel("Pérdida")
plt.title("Curva de pérdida")
plt.legend()
plt.grid(True)
plt.show()

plt.figure(figsize=(6,4))
plt.plot(epochs, history["train_acc"], label="Train Acc")
plt.plot(epochs, history["val_acc"],   label="Val Acc")
plt.xlabel("Época")
plt.ylabel("Precisión")
plt.title("Curva de precisión")
plt.legend()
plt.grid(True)
plt.show()



## (Opcional) Barrido de configuraciones de la CNN

Ejemplo de cómo probar distintas combinaciones de `kernel_size`, `stride`, `padding` y `pooling` de forma breve (1–2 épocas)  
para **comparar tamaños/precisiones** sin entrenar largo.


In [ ]:

configs = [
    dict(k1=3,s1=1,p1=1,k2=3,s2=1,p2=1, pool_kernel=2, pool_stride=2, use_pool=True),
    dict(k1=5,s1=1,p1=2,k2=3,s2=1,p2=1, pool_kernel=2, pool_stride=2, use_pool=True),
    dict(k1=3,s1=2,p1=1,k2=3,s2=2,p2=1, pool_kernel=2, pool_stride=2, use_pool=False),
    dict(k1=5,s1=2,p1=2,k2=5,s2=1,p2=2, pool_kernel=4, pool_stride=4, use_pool=True),
]

quick_results = []

for cfg in configs:
    m = SimpleCNN(**cfg).to(DEVICE)
    opt = torch.optim.Adam(m.parameters(), lr=1e-3)
    lf  = nn.CrossEntropyLoss()

    # Entrenamiento rápido (1 época para demo)
    tr_loss, tr_acc = train_one_epoch(m, train_loader, opt, lf, DEVICE)
    va_loss, va_acc = evaluate(m, val_loader, lf, DEVICE)

    # calcular tamaño de salida del extractor de características para documentar
    h = w = 28
    h = (h + 2*cfg["p1"] - cfg["k1"])//cfg["s1"] + 1
    w = (w + 2*cfg["p1"] - cfg["k1"])//cfg["s1"] + 1
    if cfg["use_pool"]:
        h = (h - cfg["pool_kernel"])//cfg["pool_stride"] + 1
        w = (w - cfg["pool_kernel"])//cfg["pool_stride"] + 1
    h = (h + 2*cfg["p2"] - cfg["k2"])//cfg["s2"] + 1
    w = (w + 2*cfg["p2"] - cfg["k2"])//cfg["s2"] + 1
    if cfg["use_pool"]:
        h = (h - cfg["pool_kernel"])//cfg["pool_stride"] + 1
        w = (w - cfg["pool_kernel"])//cfg["pool_stride"] + 1

    feat_dim = cfg["c2_out"] if "c2_out" in cfg else 64
    feat_dim *= h*w

    quick_results.append({
        "cfg": cfg,
        "train_loss": tr_loss, "train_acc": tr_acc,
        "val_loss": va_loss, "val_acc": va_acc,
        "feat_HxW": (h, w), "feat_dim": feat_dim
    })

quick_results



## Notas para evitar sobreajuste (overfitting)

- Usar **data augmentation** (ya aplicado).  
- Añadir **Dropout** entre capas fully-connected o tras convoluciones.  
- Ajustar **weight decay** en Adam (`Adam(..., weight_decay=1e-4)`).  
- Usar **early stopping** con validación o reducir la capacidad de la red.  
- Aumentar el tamaño del conjunto de datos de entrenamiento.
